## Data 
* Data from https://opendata.dwd.de/climate_environment/health/historical_alerts/heat_warnings/
* Geodata from © GeoBasis-DE / BKG, Statistisches Bundesamt (Destatis) (2022)

In [1]:
using DataFrames
using CSV
using GLMakie,GeoMakie

In [2]:
import GeoDataFrames

In [3]:
boundary=GeoDataFrames.read("Kreisgrenzen_2021_mit_Einwohnerzahl.geojson");
first(boundary,1)

Row,geometry,OBJECTID,OBJID,BEGINN,AGS,GEN,BEZ,IBZ,BEM,SN_L,SN_R,SN_K,SN_V1,SN_V2,SN_G,FK_S3,NUTS,WSK,EWZ,KFL,SHAPE_Length,SHAPE_Area
,IGeometr…,Int32,String,DateTime,String,String,String,Int32,String,String,String,String,String,String,String,String,String,DateTime,Int32,Float64,Float64,Float64
1,Geometry: wkbMultiPolygon,1,DEBKGVG200000C75,2021-12-02T00:00:00,01001,Flensburg,Kreisfreie Stadt,40,--,01,0,01,00,00,000,R,DEF01,2008-01-01T00:00:00,91113,56.73,0.527766,0.00687941


In [48]:
for year ∈ 2008:2023
    df = CSV.File("heat_warnings/heat_alerts_$year.csv") |> DataFrame;
    heat_alerts=combine(groupby(df, :Name),nrow)
    colors=[]
    for bound ∈ eachrow(boundary)
        t=findfirst(contains.(heat_alerts.Name,bound.GEN))
        if isnothing(t)
            push!(colors,0)
        else
        push!(colors,heat_alerts[t,:nrow])
        end
    end
    f=Figure()
ax=GeoAxis(f[1,1], latlims=(46.9,55.1),lonlims=(5.5,15.5))
p=poly!(ax,GeoMakie.geo2basic.(boundary.geometry),color=colors, strokewidth=0.1, strokecolor=:black,colormap=:lajolla, colorrange=(1,25),highclip=:yellow)
Colorbar(f[1,2],p,label="Heat alerts per county in year $(year)")
hidedecorations!(ax)
hidespines!(ax)
save("heat_alerts_sum_ldk_$(year).png",f)
end